In [1]:
from transformers import BertTokenizer, TFBertModel
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
bert_model = TFBertModel.from_pretrained('bert-base-cased')
import numpy as np
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.layers import Embedding, Input, Dense, Lambda
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K
import tensorflow_datasets as tfds
import pandas as pd

import sklearn as sk
import os
import nltk
from nltk.data import find

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import re
from sklearn.utils import class_weight

2022-11-29 18:30:36.794406: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-29 18:30:36.873969: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-11-29 18:30:37.307573: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-11-29 18:30:37.307647: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] 

In [16]:
def get_bert_encoding(text):
    tokenized_output = bert_tokenizer([text],
        max_length=20,
        truncation=True,
        padding='max_length', 
        return_tensors='tf')
    
    bert_encoding = bert_model(
        tokenized_output
    )
    
    return bert_encoding
test_output = get_bert_encodings("TYLER WAS HERE")

In [12]:
import pandas as pd
import numpy as np
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score

# https://medium.com/@bedigunjit/simple-guide-to-text-classification-nlp-using-svm-and-naive-bayes-with-python-421db3a72d34
# https://en.wikipedia.org/wiki/Tf%E2%80%93idf

[nltk_data] Downloading package punkt to /home/tychi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/tychi/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/tychi/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/tychi/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /home/tychi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
from transformers import RobertaTokenizer, TFRobertaModel
roberta_tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
roberta_model = TFRobertaModel.from_pretrained('roberta-base')

Downloading:   0%|          | 0.00/657M [00:00<?, ?B/s]

Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


In [17]:
def get_roberta_encoding(text):
    tokenized_output = roberta_tokenizer([text],
        max_length=20,
        truncation=True,
        padding='max_length', 
        return_tensors='tf')
    
    roberta_encoding = roberta_model(
        tokenized_output
    )
    
    return roberta_encoding

In [20]:
test = get_roberta_encoding("HELLO")

In [23]:
tokenized_output = roberta_tokenizer(['hello'],
    max_length=20,
    truncation=True,
    padding='max_length', 
    return_tensors='tf')

In [26]:
tokenized_output

{'input_ids': <tf.Tensor: shape=(1, 20), dtype=int32, numpy=
array([[    0, 42891,     2,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 20), dtype=int32, numpy=
array([[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
      dtype=int32)>}

In [29]:
roberta_model({
    'input_ids': tokenized_output.input_ids,
    'attention_mask': tokenized_output.attention_mask
})

TFBaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=<tf.Tensor: shape=(1, 20, 768), dtype=float32, numpy=
array([[[-0.07116659,  0.08391161,  0.01735196, ..., -0.07516475,
         -0.07246815, -0.01147642],
        [-0.02421619, -0.21403545,  0.11993305, ..., -0.31250283,
         -0.2365877 ,  0.06255469],
        [-0.07777613,  0.08126406, -0.00137368, ..., -0.12842245,
         -0.08609022, -0.05174505],
        ...,
        [-0.11481542, -0.05369763,  0.121876  , ..., -0.00802524,
         -0.10733951,  0.1209749 ],
        [-0.11481542, -0.05369763,  0.121876  , ..., -0.00802524,
         -0.10733951,  0.1209749 ],
        [-0.11481542, -0.05369763,  0.121876  , ..., -0.00802524,
         -0.10733951,  0.1209749 ]]], dtype=float32)>, pooler_output=<tf.Tensor: shape=(1, 768), dtype=float32, numpy=
array([[ 0.00656117, -0.22746554, -0.22742   , -0.09879842,  0.11233532,
         0.20325296,  0.27274558, -0.08799211, -0.07574458, -0.17763238,
         0.21580641, -0.016